## Cartesian product

In [1]:
# Cartesian product
'''
itertool.product(*iterables, repeat)
= ((x,y) for x in A for y in B)
'''
def product(*iterables, repeat =1):     #phải sử dụng *arg để toàn bộ iterables truyền vào được đưa thành 1 tuple
  if repeat < 0:
    raise ValueError
  pools = [tuple(pool) for pool in iterables] * repeat  #biến iterables thành list*repeat để compute

  result = [[]]
  for pool in pools:
    result = [x + [y] for x in result for y in pool]

  final_list = [tuple(prob) for prob in result]

  return final_list

product(('A','B','C'), repeat=2)
#lưu ý: iterables trong trường hợp này tương ứng với 1 vector ('A','B','C') chứ không phải 3 vector

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [ ]:
# ví dụ để hiểu đúng về vector dùng trong iterables. nếu iterables = ('A','B','C'), kết quả là [('A', 'B', 'C', 'A', 'B', 'C')]
iterables = (('A','B','C'),)
repeat = 2
pools = [tuple(pool) for pool in iterables] * repeat
# pools = [('A', 'B', 'C'), ('A', 'B', 'C')]

result = [[]]
for pool in pools:
  result = [x + [y] for x in result for y in pool]
# result giống final_list nhưng phần tử bên trong là list, không phải tuple

final_list = [tuple(prob) for prob in result]
result

[['A', 'A'],
 ['A', 'B'],
 ['A', 'C'],
 ['B', 'A'],
 ['B', 'B'],
 ['B', 'C'],
 ['C', 'A'],
 ['C', 'B'],
 ['C', 'C']]

In [2]:
product(('A','B','C'),('x','y'),(1,2), repeat=1)

[('A', 'x', 1),
 ('A', 'x', 2),
 ('A', 'y', 1),
 ('A', 'y', 2),
 ('B', 'x', 1),
 ('B', 'x', 2),
 ('B', 'y', 1),
 ('B', 'y', 2),
 ('C', 'x', 1),
 ('C', 'x', 2),
 ('C', 'y', 1),
 ('C', 'y', 2)]

In [8]:
#minh họa cho list comprehension đã xảy ra
iterables = (('A','B','C'),('x','y'),(1,2))
repeat = 1
pools = [tuple(pool) for pool in iterables] * repeat
result = [[]]
for pool in pools:
  result = [x + [y] for x in result for y in pool]
  print(result)

[['A'], ['B'], ['C']]
[['A', 'x'], ['A', 'y'], ['B', 'x'], ['B', 'y'], ['C', 'x'], ['C', 'y']]
[['A', 'x', 1], ['A', 'x', 2], ['A', 'y', 1], ['A', 'y', 2], ['B', 'x', 1], ['B', 'x', 2], ['B', 'y', 1], ['B', 'y', 2], ['C', 'x', 1], ['C', 'x', 2], ['C', 'y', 1], ['C', 'y', 2]]


## Permutation

In [ ]:
# cách 1: sử dụng generator yield
# thuật toán Steinhaus–Johnson–Trotter
def _permutations_generator(iterable, r=None):
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    if r > n:
        return

    indices = list(range(n))
    cycles = list(range(n, n - r, -1))
    yield tuple(pool[i] for i in indices[:r]) # yield tổ hợp đầu tiên

    while n:
        for i in reversed(range(r)):
            cycles[i] -= 1
            if cycles[i] == 0:
                # Xoay các chỉ mục
                indices[i:] = indices[i+1:] + indices[i:i+1]
                cycles[i] = n - i
            else:
                j = cycles[i]
                # Hoán đổi phần tử
                indices[i], indices[-j] = indices[-j], indices[i]
                yield tuple(pool[i] for i in indices[:r]) # yield tổ hợp mới
                break
        else:
            return

def permutations_list(iterable, r=None):
    # Gọi hàm generator và dùng list() để thu thập tất cả kết quả
    return list(_permutations_generator(iterable, r))

In [9]:
# Cách 2: dùng recursion - dễ hiểu hơn nhưng tốc độ chậm hơn cách 1
def permutations_recursive(iterable, r=None):
    pool = list(iterable)
    n = len(pool)
    r = n if r is None else r

    if r > n or r < 0:
        return []

    # Case cơ bản: Khi r=0, chỉ có 1 hoán vị là tuple rỗng
    if r == 0:
        return [tuple()]

    results = []

    # Lặp qua từng phần tử trong pool
    for i in range(n):
        # Chọn phần tử hiện tại
        current_element = pool[i]

        # Tạo pool còn lại (loại bỏ phần tử hiện tại, vì đây là Hoán vị không lặp)
        remaining_pool = pool[:i] + pool[i+1:]

        # Gọi đệ quy để tìm hoán vị của pool còn lại với độ dài r-1
        # Lưu ý: nếu r là n, thì r-1 là n-1, nếu r < n, thì vẫn là r-1
        sub_permutations = permutations_recursive(remaining_pool, r - 1)

        # Nối phần tử hiện tại vào tất cả các kết quả con
        for sub_p in sub_permutations:
            results.append((current_element,) + sub_p)

    return results

In [14]:
print(permutations_recursive('ABCD', r=2))

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'A'), ('B', 'C'), ('B', 'D'), ('C', 'A'), ('C', 'B'), ('C', 'D'), ('D', 'A'), ('D', 'B'), ('D', 'C')]


## Combination

In [ ]:
# Cách 1: sử dụng generator
def _combinations_generator(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    # Kiểm tra điều kiện đầu vào
    if r > n or r < 0:
        return

    # Khởi tạo các chỉ mục cho tổ hợp đầu tiên (0, 1, 2, ..., r-1)
    indices = list(range(r))

    # Yield tổ hợp đầu tiên
    yield tuple(pool[i] for i in indices)

    while True:
        # 1. Tìm vị trí 'i' xa nhất về bên phải có thể tăng lên
        # Duyệt ngược từ phải sang trái (i = r-1 đến 0)
        for i in reversed(range(r)):
            # Điều kiện kiểm tra xem chỉ mục indices[i] đã đạt đến giá trị tối đa
            # mà nó có thể đạt được: i + (n - r).
            # Tức là: indices[i] đã ở vị trí cuối cùng có thể.
            if indices[i] != i + n - r:
                break
        else:
            # Nếu vòng lặp for chạy hết (tất cả các chỉ mục đã đạt giới hạn),
            # nghĩa là đã tạo hết tất cả tổ hợp.
            return

        # 2. Tăng chỉ mục 'i'
        indices[i] += 1

        # 3. Đặt lại các chỉ mục bên phải 'i'
        # Tất cả các chỉ mục 'j' sau 'i' phải là giá trị liên tiếp tăng dần
        # bắt đầu từ indices[i] + 1.
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1

        # 4. Yield tổ hợp mới
        yield tuple(pool[i] for i in indices)

def combinations_list(iterable, r):
    """
    Hàm tính Tổ hợp và trả về danh sách (list) các kết quả.
    """
    # Gọi hàm generator và dùng list() để thu thập tất cả kết quả
    return list(_combinations_generator(iterable, r))

In [15]:
# Cách 2: sử dụng recursion
def combinations_recursive(iterable, r):
    pool = tuple(iterable)
    n = len(pool)

    # 1. Hàm đệ quy thực tế
    def generate(start_index, current_combo):
        # Trường hợp cơ sở (Base Case): Đã đủ r phần tử
        if len(current_combo) == r:
            results.append(tuple(current_combo))
            return

        # Trường hợp dừng (Stopping Condition): Không còn đủ phần tử để đạt r
        if start_index == n:
            return

        # 2. Vòng lặp chính: Duyệt qua các phần tử bắt đầu từ 'start_index'
        for i in range(start_index, n):
            # i) Tạo tổ hợp mới
            new_combo = current_combo + [pool[i]]

            # ii) Gọi đệ quy cho các phần tử còn lại
            # Bắt đầu tìm kiếm từ i + 1 để đảm bảo không bị trùng lặp
            generate(i + 1, new_combo)

    results = []
    # Bắt đầu đệ quy: tìm kiếm từ đầu (chỉ mục 0) với tổ hợp rỗng
    generate(0, [])

    return results

In [16]:
data = 'ABCD'
k = 2
result = combinations_recursive(data, k)
print(result)

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]
